<h1>TP2: Traduction automatique</h1>

In [2]:
!pip install OpenNMT-py

Defaulting to user installation because normal site-packages is not writeable


**Question 1**:  les fichiers IWSLT10_BTEC.train.en.txt et IWSLT10_BTEC.train.fr.txt contiennent les mêmes phrases, l'une en anglais et l'autre en français.

In [4]:
%%bash
awk -F '\' '{print $NF}' ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt > ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.clean.txt
awk -F '\' '{print $NF}' ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt > ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.clean.txt
awk -F '\' '{print $NF}' ./BTEC-en-fr/train/IWSLT10_BTEC.train.en.txt > ./BTEC-en-fr/train/IWSLT10_BTEC.train.en.clean.txt
awk -F '\' '{print $NF}' ./BTEC-en-fr/train/IWSLT10_BTEC.train.fr.txt > ./BTEC-en-fr/train/IWSLT10_BTEC.train.fr.clean.txt
awk -F '\' '{print $NF}' ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.txt > ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.clean.txt
awk -F '\' '{print $NF}' ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.txt > ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.clean.txt

perl tokenizer.perl -l en -lc < ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.clean.txt > ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt
perl tokenizer.perl -l fr -lc < ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.clean.txt > ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt
perl tokenizer.perl -l en -lc < ./BTEC-en-fr/train/IWSLT10_BTEC.train.en.clean.txt > ./BTEC-en-fr/train/IWSLT10_BTEC.train.en.tok.txt
perl tokenizer.perl -l fr -lc < ./BTEC-en-fr/train/IWSLT10_BTEC.train.fr.clean.txt > ./BTEC-en-fr/train/IWSLT10_BTEC.train.fr.tok.txt
perl tokenizer.perl -l fr -lc < ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.clean.txt > ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt
perl tokenizer.perl -l en -lc < ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.clean.txt > ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt

Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: fr
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: fr
Number of threads: 1
Tokenizer Version 1.1
Language: fr
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1


In [ ]:
# - - Create directory for models and predictions - -
mkdir models
mkdir models/base
mkdir BTEC-en-fr/predictions

**Question 2**: Des espaces ont été ajoutés entre les mots et les points. Les apostrophes ont été remplacés par &apos;. L'intérêt de l'opération effectué est d'avoir des fichiers plus lisible par le programme.

**Question 3**: L'ensemble train est utilisé pour former le modèle. L'ensemble de test est ensuite utilisé pour évaluer ses performances. L'ensemble développement est utilisé pour évaluer les performances finales du modèle, après que les étapes d'entraînement et de test aient été effectuées. Il simule les conditions du monde réel et mesure la capacité du modèle à généraliser sur de nouvelles données qu'il n'a pas vues auparavant.

**Question 4**: Un système de TA a besoin d'un vocabulaire pour segmenter les textes en tokens afin de faciliter la construction du modèle et réduire la complexité du modèle. On utilise pas un vocabulaire universel parce qu'on veut s'adapter à nos données, réduire la dimension du vocabulaire, et avoir un modèle plus spécialisé.

In [7]:
!onmt_build_vocab -config config-base.yaml -n_sample -1

Corpus train's weight should be given. We default it to 1 for you.
[2024-03-11 09:41:01,055 INFO] Counter vocab from -1 samples.
[2024-03-11 09:41:01,055 INFO] n_sample=-1: Build vocab on full datasets.
[2024-03-11 09:41:01,406 INFO] Counters src: 9978
[2024-03-11 09:41:01,406 INFO] Counters tgt: 8194


**Question 5**: Ces nombres nous indiquent le nombre de tokens uniques dans le corpus source (src) et dans le corpus cible (tgt)

In [10]:
!onmt_train -config config-base.yaml

[2024-03-11 09:50:23,398 INFO] Missing transforms field for train data, set to default: [].
[2024-03-11 09:50:23,398 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2024-03-11 09:50:23,398 INFO] Missing transforms field for valid data, set to default: [].
[2024-03-11 09:50:23,398 INFO] Parsed 2 corpora from -data.
[2024-03-11 09:50:23,398 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2024-03-11 09:50:23,420 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', '.', '?', 'de', ',', 'vous', 'Je']
[2024-03-11 09:50:23,420 INFO] The decoder start token is: <s>
[2024-03-11 09:50:23,420 INFO] Building model...
/home/puguix/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


**Question 6**:  
acc correspond à l'accuracy, plus cette valeur est élevée, mieux c'est car le modèle est donc plus précis dans ses prédictions.  
ppl correspond à la perplexity, plus cette valeur est faible, mieux c'est car cela signifie que le modèle a une meilleure compréhension des données.

**Question 7**:  
train_step: nombre d'étapes d'entrainement  
valid_step: fréquence à laquelle l'évaluation sur les données de validation est effectuée  
enc_layers: nombre de couches dans l'encodeur  
dec_layers: nombre de couches dans le décodeur  
enc_rnn_size: nombre de neuronnes par couche dans l'encodeur  
dec_rnn_size: nombre de neuronnes par couche dans le décodeur  
batch_size: nombre de données utilisées par pas d'entrainement

In [21]:

!onmt_translate -model ./models/base/model_step_625.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_625.txt
!onmt_translate -model ./models/base/model_step_1250.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_1250.txt
!onmt_translate -model ./models/base/model_step_1875.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_1875.txt
!onmt_translate -model ./models/base/model_step_2000.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_2000.txt

[2024-03-11 10:30:12,782 INFO] Loading checkpoint from ./models/base/model_step_625.pt
/home/puguix/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[2024-03-11 10:30:13,070 INFO] Loading data into the model
[2024-03-11 10:30:14,932 INFO] PRED SCORE: -0.8312, PRED PPL: 2.30 NB SENTENCES: 469
Time w/o python interpreter load/terminate:  2.1559011936187744
[2024-03-11 10:30:17,250 INFO] Loading checkpoint from ./models/base/model_step_1250.pt
/home/puguix/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but 

**Question 8**:  
Phrase à traduire: Pouvez-vous nettoyer ma chambre ?  
Traduction model 625: Can you tell me how ?  
Traduction model 1250: Can you keep my baggage ?  
Traduction model 1875: Can you change my room?  
Traduction model 2000: Could you keep my room ?

**Question 9**:  
Un score BLEU est compris entre 0 et 100. Il faut un score le plus proche de 100 possible. Un score de 100 indique une traduction parfaite.

In [22]:
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_625.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_1250.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_1875.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_2000.txt

BLEU = 10.09, 49.3/18.1/10.3/3.5 (BP=0.752, ratio=0.778, hyp_len=2780, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 14.35, 51.2/21.2/12.6/5.8 (BP=0.857, ratio=0.866, hyp_len=3095, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 19.17, 55.0/25.5/16.1/8.6 (BP=0.912, ratio=0.916, h

**Question 10**: Le modèle ayant le meilleur score BLEU est le modèle le plus entrainé (modèle 2000).

In [23]:
!onmt_train -config ./configs/config-early-stopping.yaml

[2024-03-11 10:42:20,385 INFO] Missing transforms field for train data, set to default: [].
[2024-03-11 10:42:20,385 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2024-03-11 10:42:20,385 INFO] Missing transforms field for valid data, set to default: [].
[2024-03-11 10:42:20,385 INFO] Parsed 2 corpora from -data.
[2024-03-11 10:42:20,385 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2024-03-11 10:42:20,407 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', '.', '?', 'de', ',', 'vous', 'Je']
[2024-03-11 10:42:20,408 INFO] The decoder start token is: <s>
[2024-03-11 10:42:20,408 INFO] Building model...
/home/puguix/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [24]:
!onmt_translate -model ./models/base/model_step_5000.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_5000.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_5000.txt

[2024-03-11 11:07:41,898 INFO] Loading checkpoint from ./models/base/model_step_5000.pt
/home/puguix/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[2024-03-11 11:07:42,227 INFO] Loading data into the model
[2024-03-11 11:07:44,538 INFO] PRED SCORE: -0.4034, PRED PPL: 1.50 NB SENTENCES: 469
Time w/o python interpreter load/terminate:  2.649181842803955
BLEU = 30.31, 59.5/35.3/24.7/16.3 (BP=1.000, ratio=1.000, hyp_len=3574, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl

**Question 11**: Augmenter le nombre de steps permet d'obtenir un meilleur score jusqu'à ce que le modèle overfit. On peut donc utiliser l'option early-stopping pour arrêter l'entrainement du modèle lorsqu'il ne s'améliore pas pendant un certain nombre de validations d'affilé. Notre modèle s'améliore jusqu'à 5000 steps.

In [26]:
!onmt_train -config ./configs/config-multiple-layers.yaml
!onmt_translate -model ./models/base/model_step_2001.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_2001.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_2001.txt

[2024-03-11 11:14:57,137 INFO] Missing transforms field for train data, set to default: [].
[2024-03-11 11:14:57,138 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2024-03-11 11:14:57,138 INFO] Missing transforms field for valid data, set to default: [].
[2024-03-11 11:14:57,138 INFO] Parsed 2 corpora from -data.
[2024-03-11 11:14:57,138 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2024-03-11 11:14:57,161 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', '.', '?', 'de', ',', 'vous', 'Je']
[2024-03-11 11:14:57,161 INFO] The decoder start token is: <s>
[2024-03-11 11:14:57,161 INFO] Building model...
[2024-03-11 11:14:57,673 INFO] Switching model to float32 for amp/apex_amp
[2024-03-11 11:14:57,673 INFO] Non quantized layer compute is fp32
[2024-03-11 11:14:57,675 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
     

**Question 12**: Cela permet d'obtenir de meilleurs résultats car on augmente la profondeur du réseau de neuronnes donc il est plus apte à comprendre des données plus complexes.

In [1]:
!onmt_train -config ./configs/config-more-neurons.yaml
!onmt_translate -model ./models/base/model_step_2002.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_2002.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_2002.txt

[2024-03-11 11:32:59,539 INFO] Missing transforms field for train data, set to default: [].
[2024-03-11 11:32:59,539 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2024-03-11 11:32:59,539 INFO] Missing transforms field for valid data, set to default: [].
[2024-03-11 11:32:59,540 INFO] Parsed 2 corpora from -data.
[2024-03-11 11:32:59,540 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2024-03-11 11:32:59,563 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', '.', '?', 'de', ',', 'vous', 'Je']
[2024-03-11 11:32:59,563 INFO] The decoder start token is: <s>
[2024-03-11 11:32:59,563 INFO] Building model...
/home/puguix/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


**Question 13**: Cela permet d'obtenir de meilleurs résultats car on augmente la profondeur du réseau de neuronnes donc il est plus apte à comprendre des données plus complexes.

In [2]:
!onmt_train -config ./configs/config-mlp-attention.yaml
!onmt_translate -model ./models/base/model_step_2003.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_2003.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_2003.txt

[2024-03-11 11:42:06,846 INFO] Missing transforms field for train data, set to default: [].
[2024-03-11 11:42:06,846 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2024-03-11 11:42:06,846 INFO] Missing transforms field for valid data, set to default: [].
[2024-03-11 11:42:06,846 INFO] Parsed 2 corpora from -data.
[2024-03-11 11:42:06,846 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2024-03-11 11:42:06,873 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', '.', '?', 'de', ',', 'vous', 'Je']
[2024-03-11 11:42:06,873 INFO] The decoder start token is: <s>
[2024-03-11 11:42:06,873 INFO] Building model...
/home/puguix/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


**Question 14**: On observe un gain de performance (score bleu 20 -> 21). Le mécanisme permet au modèle de se concentrer sur des parties spécifiques de la séquence d'entrée lors de la génération de la séquence de sortie. L'attention MLP est une approche plus flexible et plus expressive pour calculer les poids d'attention par rapport à d'autres méthodes d'attention.

**Question 15**: Le beam search est un algorithme qui au lieu d'explorer toutes les possibilités, n'en explore que quelques unes en faisant un choix grâce à un modèle probabilitste. La taille par défaut du beam est 30.

In [3]:
%%bash
onmt_translate -model ./models/base/model_step_2000.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_beam_1.txt -beam_size 1
onmt_translate -model ./models/base/model_step_2000.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_beam_3.txt -beam_size 3
onmt_translate -model ./models/base/model_step_2000.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_beam_7.txt -beam_size 7
onmt_translate -model ./models/base/model_step_2000.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_beam_10.txt -beam_size 10
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_beam_1.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_beam_3.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_beam_7.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_beam_10.txt

Time w/o python interpreter load/terminate:  1.3783504962921143
Time w/o python interpreter load/terminate:  1.8698782920837402
Time w/o python interpreter load/terminate:  3.0410964488983154
Time w/o python interpreter load/terminate:  4.2600226402282715
BLEU = 18.57, 49.9/22.7/14.2/7.4 (BP=1.000, ratio=1.045, hyp_len=3734, ref_len=3574)
BLEU = 20.43, 53.0/25.4/16.0/8.6 (BP=0.986, ratio=0.986, hyp_len=3524, ref_len=3574)
BLEU = 20.84, 54.2/26.4/17.1/9.4 (BP=0.951, ratio=0.952, hyp_len=3403, ref_len=3574)
BLEU = 21.14, 54.8/26.9/17.5/9.7 (BP=0.945, ratio=0.947, hyp_len=3384, ref_len=3574)


[2024-03-11 11:52:50,377 INFO] Loading checkpoint from ./models/base/model_step_2000.pt
/home/puguix/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[2024-03-11 11:52:50,771 INFO] Loading data into the model
[2024-03-11 11:52:51,749 INFO] PRED SCORE: -0.7095, PRED PPL: 2.03 NB SENTENCES: 469
[2024-03-11 11:52:53,962 INFO] Loading checkpoint from ./models/base/model_step_2000.pt
/home/puguix/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[2024-03-11 11:52:54,294 INFO] Loading data into the mod

**Question 16**: Plus la taille du beam est élevée, meilleurs sont les résultats.

In [4]:
!onmt_translate -model ./models/base/model_step_6000.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_6000.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_6000.txt

[2024-03-11 11:53:28,433 INFO] Loading checkpoint from ./models/base/model_step_6000.pt
/home/puguix/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[2024-03-11 11:53:28,780 INFO] Loading data into the model
[2024-03-11 11:53:30,897 INFO] PRED SCORE: -0.3571, PRED PPL: 1.43 NB SENTENCES: 469
Time w/o python interpreter load/terminate:  2.47044038772583
BLEU = 30.59, 63.0/37.8/26.2/16.6 (BP=0.959, ratio=0.960, hyp_len=3431, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl 

In [6]:
!onmt_train -config ./configs/config-best.yaml

[2024-03-14 12:37:40,087 INFO] Missing transforms field for train data, set to default: [].
[2024-03-14 12:37:40,087 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2024-03-14 12:37:40,087 INFO] Missing transforms field for valid data, set to default: [].
[2024-03-14 12:37:40,087 INFO] Parsed 2 corpora from -data.
[2024-03-14 12:37:40,087 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2024-03-14 12:37:40,110 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', '.', '?', 'de', ',', 'vous', 'Je']
[2024-03-14 12:37:40,111 INFO] The decoder start token is: <s>
[2024-03-14 12:37:40,111 INFO] Building model...
[2024-03-14 12:37:40,651 INFO] Switching model to float32 for amp/apex_amp
[2024-03-14 12:37:40,651 INFO] Non quantized layer compute is fp32
[2024-03-14 12:37:40,658 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
     

In [7]:
!onmt_translate -model ./models/futur/model_step_11000.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output ./BTEC-en-fr/predictions/pred_11000.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < BTEC-en-fr/predictions/pred_11000.txt

[2024-03-14 14:00:22,013 INFO] Loading checkpoint from ./models/futur/model_step_11000.pt
[2024-03-14 14:00:22,388 INFO] Loading data into the model
[2024-03-14 14:00:26,759 INFO] PRED SCORE: -0.2282, PRED PPL: 1.26 NB SENTENCES: 469
Time w/o python interpreter load/terminate:  4.753257751464844
BLEU = 39.21, 68.6/45.8/33.9/24.2 (BP=0.978, ratio=0.978, hyp_len=3497, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.



**Question 17**: Le meilleur score bleu est obtenu pour le modèle à 11000 pas avec les optimisations qu'on a vu ci-dessus.

**Question 18**: On a l'impression que le modèle s'améliore:  
Phrase à traduire: Pouvez-vous conduire plus lentement, s'il vous plaît?    
Phrase du modèle à 2000 pas: Could you call me , please ?  
Phrase du modèle à 6000 pas: Could you speak more slowly , please ? 
Phrase à traduire: Avez-vous un menu?      
Phrase du modèle à 2000 pas: Do you have a room ?  
Phrase du modèle à 6000 pas: Do you have a menu ?  
Phrase à traduire: Toutes les dix minutes.    
Phrase du modèle à 2000 pas: Let &apos;s check in Japan .  
Phrase du modèle à 6000 pas: Every ten minutes .